In [40]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
try:
    print(curr_dir)
except:
    curr_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    os.chdir(curr_dir)
    print(curr_dir)

import json
import numpy as np
import torch as th
from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.utils import safe_mean
from envs import BlendEnv, flatten_and_track_mappings, reconstruct_dict
from utils import *

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL


In [41]:
layout = "simple"
connections, action_sample = get_jsons(layout)
sources, blend, demands = get_sbp(connections)
tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [42]:
env = BlendEnv(v = True, T = 6,
                    D = 0.1, Q = 0,
                    P = 2, B = 2,
                    Z = 1, M = 2,
                    MAXFLOW = 500,
                    connections = connections,
                    action_sample = action_sample,
                    tau0 = tau0,delta0 = delta0)

In [43]:
action = {
    "source_blend": {
        "s1": {
            "j1": 1,
            "j2": 1,
            "j3": 0,
            "j4": 0
        },
        "s2": {
            "j1": 1,
            "j2": 1,
            "j3": 0,
            "j4": 0
        }
    },
    "blend_blend": {},
    "blend_demand": {
        "j1": {
            "p1": 0,
            "p2": 0
        },
        "j2": {
            "p1": 0,
            "p2": 0
        },
        "j3": {
            "p1": 0,
            "p2": 0
        },
        "j4": {
            "p1": 0,
            "p2": 0
        }
    },
    "tau": {
        "s1": 10,
        "s2": 10
    },
    "delta": {
        "p1": 0,
        "p2": 0
    }
}

In [44]:
state, info_d = env.reset()

In [45]:
info_d["dict_state"]

{'sources': {'s1': 0, 's2': 0},
 'blenders': {'j1': 0, 'j2': 0, 'j3': 0, 'j4': 0},
 'demands': {'p2': 0, 'p1': 0},
 'properties': {'j1': {'q1': 0},
  'j2': {'q1': 0},
  'j3': {'q1': 0},
  'j4': {'q1': 0}},
 'sources_avail_next_0': {'s1': 10, 's2': 10},
 'demands_avail_next_0': {'p2': 0, 'p1': 0},
 'sources_avail_next_1': {'s1': 10, 's2': 10},
 'demands_avail_next_1': {'p2': 0, 'p1': 0},
 'sources_avail_next_2': {'s1': 10, 's2': 10},
 'demands_avail_next_2': {'p2': 0, 'p1': 0},
 'sources_avail_next_3': {'s1': 0, 's2': 0},
 'demands_avail_next_3': {'p2': 10, 'p1': 10},
 'sources_avail_next_4': {'s1': 0, 's2': 0},
 'demands_avail_next_4': {'p2': 10, 'p1': 10},
 'sources_avail_next_5': {'s1': 0, 's2': 0},
 'demands_avail_next_5': {'p2': 10, 'p1': 10},
 't': 0}

In [46]:
act_tensor = flatten_and_track_mappings(action)[0]

In [47]:
n_state, rew, term, trunc, d_info = env.step(act_tensor)

[INFO] Increased reward by 0.8 through tank population in s1
[INFO] Increased reward by 0.8 through tank population in s2
j1: inv: 0, in_flow_sources: 2.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
[INFO] Increased reward by 0.4 through tank population in j1
j2: inv: 0, in_flow_sources: 2.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
[INFO] Increased reward by 0.4 through tank population in j2
j3: inv: 0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j4: inv: 0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0


In [48]:
action = {
    "source_blend": {
        "s1": {
            "j1": 1,
            "j2": 1,
            "j3": 0,
            "j4": 0
        },
        "s2": {
            "j1": 1,
            "j2": 1,
            "j3": 0,
            "j4": 0
        }
    },
    "blend_blend": {},
    "blend_demand": {
        "j1": {
            "p1": 0,
            "p2": 0
        },
        "j2": {
            "p1": 0,
            "p2": 0
        },
        "j3": {
            "p1": 0,
            "p2": 0
        },
        "j4": {
            "p1": 0,
            "p2": 0
        }
    },
    "tau": {
        "s1": 12,
        "s2": 10
    },
    "delta": {
        "p1": 0,
        "p2": 0
    }
}

In [49]:
info_d["dict_state"]

{'sources': {'s1': 8.0, 's2': 8.0},
 'blenders': {'j1': 2.0, 'j2': 2.0, 'j3': 0.0, 'j4': 0.0},
 'demands': {'p2': 0.0, 'p1': 0.0},
 'properties': {'j1': {'q1': 0.16},
  'j2': {'q1': 0.16},
  'j3': {'q1': 0},
  'j4': {'q1': 0}},
 'sources_avail_next_0': {'s1': 10, 's2': 10},
 'demands_avail_next_0': {'p2': 0, 'p1': 0},
 'sources_avail_next_1': {'s1': 10, 's2': 10},
 'demands_avail_next_1': {'p2': 0, 'p1': 0},
 'sources_avail_next_2': {'s1': 0, 's2': 0},
 'demands_avail_next_2': {'p2': 10, 'p1': 10},
 'sources_avail_next_3': {'s1': 0, 's2': 0},
 'demands_avail_next_3': {'p2': 10, 'p1': 10},
 'sources_avail_next_4': {'s1': 0, 's2': 0},
 'demands_avail_next_4': {'p2': 10, 'p1': 10},
 'sources_avail_next_5': {'s1': 0, 's2': 0},
 'demands_avail_next_5': {'p2': 0, 'p1': 0},
 't': 1}

In [50]:
act_tensor = flatten_and_track_mappings(action)[0]
n_state, rew, term, trunc, d_info = env.step(act_tensor)

[PEN] t2; s1:			bought too much (more than supply): 12.0 vs 10
[INFO] Increased reward by 0.8 through tank population in s1
[INFO] Increased reward by 0.8 through tank population in s2
j1: inv: 2.0, in_flow_sources: 2.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
[INFO] Increased reward by 0.4 through tank population in j1
j2: inv: 2.0, in_flow_sources: 2.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
[INFO] Increased reward by 0.4 through tank population in j2
j3: inv: 0.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j4: inv: 0.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0


In [51]:
d_info["pen_tracker"]

{'M': 0,
 'B': -4.0,
 'P': 0,
 'Q': 0,
 'n_M': 0,
 'n_B': 1,
 'n_P': 0,
 'n_Q': 0,
 'reg': 0,
 'units_sold': 0,
 'units_bought': 40.0,
 'rew_sold': 0,
 'rew_depth': 4.800000000000001}

In [52]:
action = {
    "source_blend": {
        "s1": {
            "j1": 1,
            "j2": 1,
            "j3": 0,
            "j4": 0
        },
        "s2": {
            "j1": 1,
            "j2": 1,
            "j3": 0,
            "j4": 0
        }
    },
    "blend_blend": {},
    "blend_demand": {
        "j1": {
            "p1": 1,
            "p2": 1
        },
        "j2": {
            "p1": 0,
            "p2": 0
        },
        "j3": {
            "p1": 0,
            "p2": 0
        },
        "j4": {
            "p1": 0,
            "p2": 0
        }
    },
    "tau": {
        "s1": 0,
        "s2": 0
    },
    "delta": {
        "p1": 0,
        "p2": 0
    }
}

In [53]:
info_d["dict_state"]


{'sources': {'s1': 16.0, 's2': 16.0},
 'blenders': {'j1': 4.0, 'j2': 4.0, 'j3': 0.0, 'j4': 0.0},
 'demands': {'p2': 0.0, 'p1': 0.0},
 'properties': {'j1': {'q1': 0.24},
  'j2': {'q1': 0.24},
  'j3': {'q1': 0},
  'j4': {'q1': 0}},
 'sources_avail_next_0': {'s1': 10, 's2': 10},
 'demands_avail_next_0': {'p2': 0, 'p1': 0},
 'sources_avail_next_1': {'s1': 0, 's2': 0},
 'demands_avail_next_1': {'p2': 10, 'p1': 10},
 'sources_avail_next_2': {'s1': 0, 's2': 0},
 'demands_avail_next_2': {'p2': 10, 'p1': 10},
 'sources_avail_next_3': {'s1': 0, 's2': 0},
 'demands_avail_next_3': {'p2': 10, 'p1': 10},
 'sources_avail_next_4': {'s1': 0, 's2': 0},
 'demands_avail_next_4': {'p2': 0, 'p1': 0},
 'sources_avail_next_5': {'s1': 0, 's2': 0},
 'demands_avail_next_5': {'p2': 0, 'p1': 0},
 't': 2}

In [54]:
act_tensor = flatten_and_track_mappings(action)[0]
n_state, rew, term, trunc, d_info = env.step(act_tensor)


j1: inv: 4.0, in_flow_sources: 2.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 2.0
[PEN] t3; j1:			In and out flow both non-zero (in: 2.0, out: 2.0)
j2: inv: 4.0, in_flow_sources: 2.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
[INFO] Increased reward by 0.4 through tank population in j2
j3: inv: 0.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j4: inv: 0.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0


In [55]:
d_info["pen_tracker"]

{'M': -2,
 'B': -4.0,
 'P': 0,
 'Q': 0,
 'n_M': 1,
 'n_B': 1,
 'n_P': 0,
 'n_Q': 0,
 'reg': 0,
 'units_sold': 0,
 'units_bought': 40.0,
 'rew_sold': 0,
 'rew_depth': 5.200000000000001}

In [56]:
action = {
    "source_blend": {
        "s1": {
            "j1": 0,
            "j2": 0,
            "j3": 0,
            "j4": 0
        },
        "s2": {
            "j1": 0,
            "j2": 0,
            "j3": 0,
            "j4": 0
        }
    },
    "blend_blend": {},
    "blend_demand": {
        "j1": {
            "p1": 100,
            "p2": 1
        },
        "j2": {
            "p1": 0,
            "p2": 0
        },
        "j3": {
            "p1": 0,
            "p2": 0
        },
        "j4": {
            "p1": 0,
            "p2": 0
        }
    },
    "tau": {
        "s1": 0,
        "s2": 0
    },
    "delta": {
        "p1": 0,
        "p2": 0
    }
}

In [57]:
d_info["dict_state"]

{'sources': {'s1': 14.0, 's2': 14.0},
 'blenders': {'j1': 4.0, 'j2': 6.0, 'j3': 0.0, 'j4': 0.0},
 'demands': {'p2': 0.0, 'p1': 0.0},
 'properties': {'j1': {'q1': 0.24},
  'j2': {'q1': 0.29333333333333333},
  'j3': {'q1': 0},
  'j4': {'q1': 0}},
 'sources_avail_next_0': {'s1': 0, 's2': 0},
 'demands_avail_next_0': {'p2': 10, 'p1': 10},
 'sources_avail_next_1': {'s1': 0, 's2': 0},
 'demands_avail_next_1': {'p2': 10, 'p1': 10},
 'sources_avail_next_2': {'s1': 0, 's2': 0},
 'demands_avail_next_2': {'p2': 10, 'p1': 10},
 'sources_avail_next_3': {'s1': 0, 's2': 0},
 'demands_avail_next_3': {'p2': 0, 'p1': 0},
 'sources_avail_next_4': {'s1': 0, 's2': 0},
 'demands_avail_next_4': {'p2': 0, 'p1': 0},
 'sources_avail_next_5': {'s1': 0, 's2': 0},
 'demands_avail_next_5': {'p2': 0, 'p1': 0},
 't': 3}

In [58]:
act_tensor = flatten_and_track_mappings(action)[0]
n_state, rew, term, trunc, d_info = env.step(act_tensor)

j1: inv: 4.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 101.0
[PEN] t4; j1:			inventory OOB (resulting amount less than blending tank LB)
j1: b: 0.039603960396039604
j2: inv: 6.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j3: inv: 0.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j4: inv: 0.0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
[INFO] Increased reward by 0.015841584158415842 through tank population in p2
[INFO] Increased reward by 1.5841584158415842 through tank population in p1


In [60]:
d_info["dict_state"]

{'sources': {'s1': 14.0, 's2': 14.0},
 'blenders': {'j1': 0.0, 'j2': 6.0, 'j3': 0.0, 'j4': 0.0},
 'demands': {'p2': 0.039603960396039604, 'p1': 3.9603960396039604},
 'properties': {'j1': {'q1': 0},
  'j2': {'q1': 0.29333333333333333},
  'j3': {'q1': 0},
  'j4': {'q1': 0}},
 'sources_avail_next_0': {'s1': 0, 's2': 0},
 'demands_avail_next_0': {'p2': 10, 'p1': 10},
 'sources_avail_next_1': {'s1': 0, 's2': 0},
 'demands_avail_next_1': {'p2': 10, 'p1': 10},
 'sources_avail_next_2': {'s1': 0, 's2': 0},
 'demands_avail_next_2': {'p2': 0, 'p1': 0},
 'sources_avail_next_3': {'s1': 0, 's2': 0},
 'demands_avail_next_3': {'p2': 0, 'p1': 0},
 'sources_avail_next_4': {'s1': 0, 's2': 0},
 'demands_avail_next_4': {'p2': 0, 'p1': 0},
 'sources_avail_next_5': {'s1': 0, 's2': 0},
 'demands_avail_next_5': {'p2': 0, 'p1': 0},
 't': 4}